# Shap Regression

In [1]:
from BorutaShap import BorutaShap, load_data

    
X, y = load_data(data_type='regression')
X.head()

C:\Users\Retro\anaconda3\envs\boruta_test1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [ ]:
# no model selected default is Random Forest, if classification is False it is a Regression problem
Feature_Selector = BorutaShap(importance_measure='shap',
                              classification=False)

Feature_Selector.fit(X=X, y=y, n_trials=100, random_state=0)

  8%|█████████████▊                                                                                                                                                               | 8/100 [01:00<09:56,  6.48s/it]

In [ ]:
# Returns Boxplot of features
# I would recommend plotting Y axis on the Log Scale
Feature_Selector.plot(X_size=12, figsize=(12,8),
            y_scale='log', which_features='all')

In [ ]:
subset = Feature_Selector.Subset()
subset.head()

# Gini Regression

In [ ]:
# no model selected default is Random Forest, if classification is False it is a Regression problem
Feature_Selector = BorutaShap(importance_measure='gini',
                              classification=False)

Feature_Selector.fit(X=X, y=y, n_trials=100, random_state=0)

In [ ]:
# Returns Boxplot of features
# I would recommend plotting Y axis on the Log Scale
Feature_Selector.plot(X_size=12, figsize=(12,8),
            y_scale='log', which_features='all')

# Shap Regression Sampling

In [ ]:
# no model selected default is Random Forest, if classification is False it is a Regression problem
Feature_Selector = BorutaShap(importance_measure='shap',
                              classification=False)

Feature_Selector.fit(X=X, y=y, n_trials=100, random_state=0, sample=True)

In [ ]:
# Returns Boxplot of features
# I would recommend plotting Y axis on the Log Scale
Feature_Selector.plot(X_size=12, figsize=(12,8),
            y_scale='log', which_features='all')

In [ ]:
y_np = np.array(y)
y_np

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np


tscv = TimeSeriesSplit(n_splits=2, test_size = int(np.floor(X.shape[0]*0.1)))
train_index, test_index = list(tscv.split(X))[1]
X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index], y_np[train_index]

In [ ]:
class CustomTimeSeriesSplit1:
    """
    A custom time series splitter that generates train-test splits for time series data.
    
    Attributes:
    ----------
    n_splits : int
        The number of splits to generate.
    test_size : float
        The fraction of data to be used as the test set in each split.
    step_size : float
        The fraction of data to be used as the step size for shifting the window.

    Methods:
    -------
    split(X, y=None, groups=None):
        Generates train-test indices for each split.
    """
    def __init__(self, n_splits=3, test_size=0.2, step_size=0.05):
        """
        Initializes the CustomTimeSeriesSplit with the given parameters.
        
        Parameters:
        ----------
        n_splits : int, optional
            The number of splits to generate (default is 3).
        test_size : float, optional
            The fraction of data to be used as the test set in each split (default is 0.2).
        step_size : float, optional
            The fraction of data to be used as the step size for shifting the window (default is 0.05).
        """
        self.n_splits = n_splits
        self.test_size = test_size
        self.step_size = step_size

    def split(self, X, y=None, groups=None):
        """
        Generates train-test indices for each split.
        
        Parameters:
        ----------
        X : array-like, shape (n_samples, n_features)
            The data to split.
        y : array-like, shape (n_samples,), optional
            The target variable (not used in this method).
        groups : array-like, shape (n_samples,), optional
            Group labels for the samples (not used in this method).

        Yields:
        -------
        train_idx : ndarray
            Indices for the training data.
        test_idx : ndarray
            Indices for the test data.
        """
        n_samples = len(X)
        
        # Calculate test size and step size in terms of number of samples
        n_test = builtins.max(int(n_samples * self.test_size), 1)
        step = builtins.max(int(n_samples * self.step_size), 1)
        
        # Adjust the number of splits if necessary
        max_splits = (n_samples - n_test) // step + 1
        n_splits = builtins.min(self.n_splits, max_splits)
        
        test_starts = range(n_samples - n_test, n_samples - n_test - n_splits * step, -step)
        
        indices = np.arange(n_samples)
        for test_start in test_starts:
            test_end = test_start + n_test
            train_idx = indices[:test_start]
            test_idx = indices[test_start:test_end]
            yield train_idx, test_idx


def custom_tscv_split(df,grouping_col, n_splits=3, test_size=0.2, step_size=0.05):
    """
    Splits time series data into train-test sets based on a custom time series cross-validation scheme.
    
    Parameters:
    ----------
    df : pd.DataFrame
        The DataFrame to be split.
    grouping_col : str
        The column name to group the data by (e.g., series ID).
    n_splits : int, optional
        The number of splits to generate (default is 3).
    test_size : float, optional
        The fraction of data to be used as the test set in each split (default is 0.2).
    step_size : float, optional
        The fraction of data to be used as the step size for shifting the window (default is 0.05).

    Returns:
    -------
    list of tuples
        Each tuple contains two DataFrames: the training set and the test set for a split.
    """
    tscv = CustomTimeSeriesSplit1(n_splits=n_splits, test_size=test_size, step_size=step_size)
    grouped = df.groupby(grouping_col)

    train_splits = [[] for _ in range(n_splits)]
    test_splits = [[] for _ in range(n_splits)]
    
    for series_id, group in grouped:
        group = group.reset_index(drop=True)
        n_samples = len(group)
        
        n_test = builtins.max(int(n_samples * test_size), 1)
        step = builtins.max(int(n_samples * step_size), 1)
        max_splits = (n_samples - n_test) // step + 1
        actual_splits = builtins.min(n_splits, max_splits)
        
        #if actual_splits < 1:
        if n_samples < 60:
            print(f"Series {series_id} does not have enough data for the specified number of splits. Defaulting to single train-test split.")
            train_idx = np.arange(n_samples - n_test)
            test_idx = np.arange(n_samples - n_test, n_samples)
            for i in range(n_splits):
                train_splits[i].append(group.iloc[train_idx])
                test_splits[i].append(group.iloc[test_idx])
        else:
            split_count = 0
            for train_idx, test_idx in tscv.split(group):
                if split_count < n_splits:
                    train_splits[split_count].append(group.iloc[train_idx])
                    test_splits[split_count].append(group.iloc[test_idx])
                    split_count += 1
    
    # Concatenate valid splits
    train_splits = [pd.concat(split) if split else pd.DataFrame() for split in train_splits]
    test_splits = [pd.concat(split) if split else pd.DataFrame() for split in test_splits]
    
    return list(zip(train_splits, test_splits))

In [ ]:
X['grp'] = 'all'
X.shape[0]

In [ ]:
import builtins
import pandas as pd
splits = custom_tscv_split(X, grouping_col = 'grp', n_splits=2,\
                            test_size = 0.1,step_size = 0.05)

In [ ]:
for i, (train_index, test_index) in enumerate(splits):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index.shape[0]}")
    print(f"  Test:  index={test_index.shape[0]}")